https://8weeksqlchallenge.com/case-study-2/

In [1]:
import duckdb
con = duckdb.connect(database=':memory:', read_only=False)

# Create runners table
con.execute("""
DROP TABLE IF EXISTS runners;
CREATE TABLE runners (
  runner_id INTEGER,
  registration_date DATE
);
INSERT INTO runners VALUES
  (1, '2021-01-01'),
  (2, '2021-01-03'),
  (3, '2021-01-08'),
  (4, '2021-01-15');
""")

# Create customer_orders table
con.execute("""
DROP TABLE IF EXISTS customer_orders;
CREATE TABLE customer_orders (
  order_id INTEGER,
  customer_id INTEGER,
  pizza_id INTEGER,
  exclusions VARCHAR,
  extras VARCHAR,
  order_time TIMESTAMP
);
INSERT INTO customer_orders VALUES
  (1, 101, 1, '', '', '2020-01-01 18:05:02'),
  (2, 101, 1, '', '', '2020-01-01 19:00:52'),
  (3, 102, 1, '', '', '2020-01-02 23:51:23'),
  (3, 102, 2, '', NULL, '2020-01-02 23:51:23'),
  (4, 103, 1, '4', '', '2020-01-04 13:23:46'),
  (4, 103, 1, '4', '', '2020-01-04 13:23:46'),
  (4, 103, 2, '4', '', '2020-01-04 13:23:46'),
  (5, 104, 1, NULL, '1', '2020-01-08 21:00:29'),
  (6, 101, 2, NULL, NULL, '2020-01-08 21:03:13'),
  (7, 105, 2, NULL, '1', '2020-01-08 21:20:29'),
  (8, 102, 1, NULL, NULL, '2020-01-09 23:54:33'),
  (9, 103, 1, '4', '1, 5', '2020-01-10 11:22:59'),
  (10, 104, 1, NULL, NULL, '2020-01-11 18:34:49'),
  (10, 104, 1, '2, 6', '1, 4', '2020-01-11 18:34:49');
""")

# Create runner_orders table
con.execute("""
DROP TABLE IF EXISTS runner_orders;
CREATE TABLE runner_orders (
  order_id INTEGER,
  runner_id INTEGER,
  pickup_time VARCHAR,
  distance VARCHAR,
  duration VARCHAR,
  cancellation VARCHAR
);
INSERT INTO runner_orders VALUES
  (1, 1, '2020-01-01 18:15:34', '20km', '32 minutes', ''),
  (2, 1, '2020-01-01 19:10:54', '20km', '27 minutes', ''),
  (3, 1, '2020-01-03 00:12:37', '13.4km', '20 mins', NULL),
  (4, 2, '2020-01-04 13:53:03', '23.4', '40', NULL),
  (5, 3, '2020-01-08 21:10:57', '10', '15', NULL),
  (6, 3, NULL, NULL, NULL, 'Restaurant Cancellation'),
  (7, 2, '2020-01-08 21:30:45', '25km', '25mins', NULL),
  (8, 2, '2020-01-10 00:15:02', '23.4 km', '15 minute', NULL),
  (9, 2, NULL, NULL, NULL, 'Customer Cancellation'),
  (10, 1, '2020-01-11 18:50:20', '10km', '10minutes', NULL);
""")

# Create pizza_names table
con.execute("""
DROP TABLE IF EXISTS pizza_names;
CREATE TABLE pizza_names (
  pizza_id INTEGER,
  pizza_name TEXT
);
INSERT INTO pizza_names VALUES
  (1, 'Meatlovers'),
  (2, 'Vegetarian');
""")

# Create pizza_recipes table
con.execute("""
DROP TABLE IF EXISTS pizza_recipes;
CREATE TABLE pizza_recipes (
  pizza_id INTEGER,
  toppings TEXT
);
INSERT INTO pizza_recipes VALUES
  (1, '1, 2, 3, 4, 5, 6, 8, 10'),
  (2, '4, 6, 7, 9, 11, 12');
""")

# Create pizza_toppings table
con.execute("""
DROP TABLE IF EXISTS pizza_toppings;
CREATE TABLE pizza_toppings (
  topping_id INTEGER,
  topping_name TEXT
);
INSERT INTO pizza_toppings VALUES
  (1, 'Bacon'),
  (2, 'BBQ Sauce'),
  (3, 'Beef'),
  (4, 'Cheese'),
  (5, 'Chicken'),
  (6, 'Mushrooms'),
  (7, 'Onions'),
  (8, 'Pepperoni'),
  (9, 'Peppers'),
  (10, 'Salami'),
  (11, 'Tomatoes'),
  (12, 'Tomato Sauce');
""")

## A. Pizza Metrics

In [3]:
# How many pizzas were ordered?
con.execute("SELECT COUNT(DISTINCT order_id) as pizza_orders FROM customer_orders;").fetchall()

[(10,)]

In [ ]:
# How many unique customer orders were made?
con.execute("SELECT COUNT(DISTINCT customer_id) as customers FROM customer_orders;").fetchall()

In [ ]:
# How many successful orders were delivered by each runner?
con.execute("""
            SELECT 
            runner_id, 
            COUNT(DISTINCT order_id) as orders 
            FROM runner_orders 
            WHERE pickup_time NOT NULL -- where pickup is NULL means order is cancelled
            GROUP BY runner_id;
            """).fetchall()

In [ ]:
# How many of each type of pizza was delivered?
con.execute("""
            SELECT p.pizza_name, 
            COUNT(DISTINCT c.order_id) as orders 
            FROM customer_orders c
            JOIN pizza_names p
            ON p.pizza_id = c.pizza_id
            GROUP BY p.pizza_name
            """).fetchall()

In [ ]:
# How many Vegetarian and Meatlovers were ordered by each customer?
con.execute("""
            SELECT 
            c.customer_id,
            SUM(CASE WHEN p.pizza_name = 'Vegetarian'
            THEN 1 
            ELSE 0
            END) as Vegetarian_Pizzas,
            SUM(CASE WHEN p.pizza_name = 'Meatlovers'
            THEN 1 
            ELSE 0
            END) as Meatlovers_Pizzas
            FROM customer_orders c
            JOIN pizza_names p
            ON p.pizza_id = c.pizza_id
            GROUP BY ALL
            """).fetchall()

In [ ]:
# What was the maximum number of pizzas delivered in a single order?
con.execute("""
            SELECT 
            order_id, 
            COUNT(pizza_id) as pizzas 
            FROM customer_orders
            GROUP BY order_id
            ORDER BY pizzas DESC
            LIMIT 1;
            """).fetchall()

In [ ]:
# For each customer, how many delivered pizzas had at least 1 change and how many had no changes?
con.execute(""" 
            SELECT
            customer_id,
            SUM(
            CASE WHEN exclusions IS NULL AND extras IS NULL -- no change
            THEN 1
            ELSE 0
            END
            ) as orders_with_no_change,
            SUM(
            CASE WHEN exclusions IS NOT NULL OR extras IS NOT NULL
            THEN 1
            ELSE 0
            END
            ) as orders_with_more_than_1_change
            FROM customer_orders
            GROUP BY ALL
            """).fetchall()

In [8]:
# How many pizzas were delivered that had both exclusions and extras?
con.execute(""" 
            SELECT
            COUNT( DISTINCT order_id)
            FROM customer_orders
            WHERE exclusions IS NOT NULL AND extras IS NOT NULL
            """).fetchall()

[(6,)]

In [14]:
# What was the total volume of pizzas ordered for each hour of the day?
con.execute(""" 
            SELECT
            extract('hour' FROM order_time) as hour,
            COUNT(DISTINCT order_id)
            FROM customer_orders
            GROUP BY extract('hour' FROM order_time)
            ORDER BY hour
                """).fetchall()

[(11, 1), (13, 1), (18, 2), (19, 1), (21, 3), (23, 2)]

In [15]:
# What was the volume of orders for each day of the week?
con.execute(""" 
            SELECT
            dayofweek(order_time) as day_of_week,
            COUNT(DISTINCT order_id)
            FROM customer_orders
            GROUP BY dayofweek(order_time)
            ORDER BY day_of_week
                """).fetchall()

[(3, 5), (4, 2), (5, 1), (6, 2)]

## B. Runner and Customer Experience

In [18]:
# How many runners signed up for each 1 week period? (i.e. week starts 2021-01-01)
con.execute(""" 
            SELECT
            -- week
            week(registration_date) as week,
            -- no. of runners signed up during that week
            COUNT(runner_id)
            FROM runners
            WHERE registration_date >= '2021-01-01'
            GROUP BY week(registration_date)
            ORDER BY week
            """).fetchdf()



,week,count(runner_id)
0,1,1
1,2,1
2,53,2


In [28]:
# What was the average time in minutes it took for each runner to arrive at the Pizza Runner HQ to pickup the order?
con.execute(""" 
            SELECT
            r.runner_id,
            ROUND(AVG(date_diff('minute', c.order_time, CAST(r.pickup_time AS TIMESTAMP))),1) AS avg_time_to_pickup_mins  -- total number of minutes between the two timestamps
            FROM runner_orders r
            JOIN customer_orders c
            ON r.order_id = c.order_id
            WHERE r.pickup_time IS NOT NULL
            GROUP BY r.runner_id,
                """).fetchdf()

,runner_id,avg_time_to_pickup_mins
0,1,15.7
1,2,24.2
2,3,10.0


In [10]:
# Is there any relationship between the number of pizzas and how long the order takes to prepare?
# for this we can assume that order_time is when the order was placed and pickup_time was when the order was ready

con.execute(""" 
            WITH pizzas_per_order_with_time AS (
            SELECT
            c.order_id,
            ROUND(AVG(date_diff('minute', c.order_time, CAST(r.pickup_time AS TIMESTAMP))),1) AS avg_time_to_pickup_mins,  -- total number of minutes between the two timestamps
            COUNT(*) as pizzas
            FROM customer_orders c
            JOIN runner_orders r 
            ON r.order_id = c.order_id
            WHERE r.pickup_time IS NOT NULL
            GROUP BY c.order_id
            )
            SELECT
            pizzas,
            AVG(avg_time_to_pickup_mins) as avg_order_completion_time_mins
            FROM pizzas_per_order_with_time
            GROUP BY pizzas
            ORDER BY avg_order_completion_time_mins
                """).fetchdf()

,pizzas,avg_order_completion_time_mins
0,1,12.2
1,2,18.5
2,3,30.0


In [17]:
# What was the average distance travelled for each customer?
con.execute(""" 
            SELECT
            c.customer_id,
            ROUND(AVG(CAST(REGEXP_EXTRACT(r.distance, '[0-9]+(\.[0-9]+)?') AS FLOAT)),2) AS avg_distance_km
            FROM customer_orders c
            JOIN runner_orders r 
            ON r.order_id = c.order_id
            WHERE r.pickup_time IS NOT NULL -- filter for cancelled orders
            GROUP BY c.customer_id
                """).fetchdf()

,customer_id,avg_distance_km
0,101,20.00
1,102,16.73
2,103,23.40
3,104,10.00
4,105,25.00


In [33]:
# What was the difference between the longest and shortest delivery times for all orders?
con.execute(""" 
    WITH orders_with_delivery_time AS (
        SELECT 
            order_id, 
            CAST(REGEXP_EXTRACT(duration, '[0-9]+(\.[0-9]+)?') AS FLOAT) AS duration
        FROM runner_orders
        WHERE pickup_time IS NOT NULL
    )
    SELECT 
        MAX(duration) - MIN(duration) AS delivery_time_difference
    FROM orders_with_delivery_time;
""").fetchdf()


,delivery_time_difference
0,30.0


In [36]:
# What was the average speed for each runner for each delivery and do you notice any trend for these values?

# by speed lets assume we are talking about duration
con.execute(""" 
        SELECT 
            runner_id, 
            AVG(CAST(REGEXP_EXTRACT(duration, '[0-9]+(\.[0-9]+)?') AS FLOAT)) AS avg_duration,
            COUNT(order_id) as orders
        FROM runner_orders
        WHERE pickup_time IS NOT NULL
        GROUP BY runner_id
        """).fetchdf()

,runner_id,avg_duration,orders
0,1,22.250000,4
1,2,26.666667,3
2,3,15.000000,1


## Ingredient Optimisation

In [19]:
# What are the standard ingredients for each pizza?
con.execute("""
            WITH exploded_toppings AS (
                SELECT
                    pizza_id,
                    UNNEST(STRING_SPLIT(toppings, ', '))::INTEGER AS topping_id
                FROM pizza_recipes
            ),
            common_toppings_id AS (
            SELECT
            topping_id
            FROM exploded_toppings
            GROUP BY topping_id
            HAVING COUNT(DISTINCT pizza_id) = (SELECT COUNT(DISTINCT pizza_id) FROM pizza_recipes) 
            -- get only those toppings which are present in all pizzas
            )

            SELECT
            pt.topping_name as standard_ingredient
            FROM common_toppings_id ct
            JOIN pizza_toppings pt
            ON ct.topping_id = pt.topping_id
            ORDER BY pt.topping_name;
            """).fetchdf()


,standard_ingredient
0,Cheese
1,Mushrooms


In [31]:
# What was the most commonly added extra?
con.execute(""" 
            WITH orders_with_extras_unnested AS (
            SELECT
            order_id,
            UNNEST(STRING_SPLIT(extras, ',')) as extras_topping_id
            FROM customer_orders
            WHERE extras IS NOT NULL AND extras <> ''
            ), 
            extras_with_ranks AS (
            SELECT
            extras_topping_id,
            COUNT(order_id) as orders,
            ROW_NUMBER() OVER (ORDER BY COUNT(order_id) DESC) as row_num
            FROM orders_with_extras_unnested
            GROUP BY extras_topping_id
            )

            SELECT 
            pt.topping_name
            FROM extras_with_ranks e
            JOIN pizza_toppings pt
            ON pt.topping_id = e.extras_topping_id
            WHERE row_num = 1
            """).fetch_df()

,topping_name
0,Bacon


In [33]:
# What was the most common exclusion?
con.execute(""" 
            WITH orders_with_exclusions_unnested AS (
            SELECT
            order_id,
            UNNEST(STRING_SPLIT(exclusions, ',')) as exclusions_topping_id
            FROM customer_orders
            WHERE exclusions IS NOT NULL AND exclusions <> ''
            ), 
            exclusions_with_ranks AS (
            SELECT
            exclusions_topping_id,
            COUNT(order_id) as orders,
            ROW_NUMBER() OVER (ORDER BY COUNT(order_id) DESC) as row_num
            FROM orders_with_exclusions_unnested
            GROUP BY exclusions_topping_id
            )

            SELECT 
            pt.topping_name
            FROM exclusions_with_ranks e
            JOIN pizza_toppings pt
            ON pt.topping_id = e.exclusions_topping_id
            WHERE row_num = 1
            """).fetch_df()

,topping_name
0,Cheese


In [9]:
''' Generate an order item for each record in the customers_orders table in the format of one of the following:
Meat Lovers
Meat Lovers - Exclude Beef
Meat Lovers - Extra Bacon
Meat Lovers - Exclude Cheese, Bacon - Extra Mushroom, Peppers
'''

con.execute(""" 
            WITH orders_with_pizza_names AS (
            SELECT
            co.order_id,
            co.pizza_id,
            pn.pizza_name,
            CASE 
            WHEN exclusions IS NULL OR exclusions = '' THEN NULL 
            ELSE exclusions 
            END AS clean_exclusions,
            CASE 
            WHEN extras IS NULL OR extras = '' THEN NULL 
            ELSE extras 
            END AS clean_extras
            FROM customer_orders co
            JOIN pizza_names pn
            ON pn.pizza_id = co.pizza_id
            ),

            orders_with_exclusions_extras AS (
            SELECT
            order_id,
            pizza_name,
            pt_exclusions.topping_name as excluded_topping,
            pt_extras.topping_name as extra_topping
            FROM orders_with_pizza_names
            LEFT JOIN pizza_toppings pt_exclusions 
            ON CONTAINS(STRING_SPLIT(clean_exclusions, ','), pt_exclusions.topping_id::STRING)
            LEFT JOIN pizza_toppings pt_extras
            ON CONTAINS(STRING_SPLIT(clean_extras, ','), pt_extras.topping_id::STRING)
            ),
            orders_grouped AS (
            SELECT 
            order_id,
            pizza_name,
            STRING_AGG(excluded_topping, ', ') as excluded_toppings,
            STRING_AGG(extra_topping, ', ') as extra_toppings
            FROM orders_with_exclusions_extras
            GROUP BY order_id, pizza_name
            )
            SELECT
            order_id,
            CASE
            -- when there are no changes
            WHEN excluded_toppings IS NULL AND extra_toppings IS NULL THEN pizza_name
            -- when there is only extras added
            WHEN excluded_toppings IS NULL AND extra_toppings IS NOT NULL
            THEN pizza_name || ' - Extras ' || extra_toppings
            -- when there is only exclusions added
            WHEN excluded_toppings IS NOT NULL AND extra_toppings IS NULL
            THEN pizza_name || ' - Excluded ' || excluded_toppings
            -- when there are both exclusions and extras
            ELSE pizza_name || ' - Excluded ' || excluded_toppings || ' - Extras ' || extra_toppings
            END as pizza_name_with_details
            FROM orders_grouped
            ORDER BY order_id
                """).fetch_df()

,order_id,pizza_name_with_details
0,1,Meatlovers
1,2,Meatlovers
2,3,Meatlovers
3,3,Vegetarian
4,4,"Meatlovers - Excluded Cheese, Cheese"
5,4,Vegetarian - Excluded Cheese
6,5,Meatlovers - Extras Bacon
7,6,Vegetarian
8,7,Vegetarian - Extras Bacon
9,8,Meatlovers


In [108]:
'''
Generate an alphabetically ordered comma separated ingredient list for each pizza order 
from the customer_orders table and add a 2x in front of any relevant ingredients
For example: "Meat Lovers: 2xBacon, Beef, ... , Salami"
'''

con.execute(""" 
            WITH 
            base_toppings AS (
            SELECT 
            co.order_id,
            co.pizza_id,
            UNNEST(STRING_SPLIT(pr.toppings, ','))::INTEGER as topping_id
            FROM customer_orders co
            JOIN pizza_recipes pr ON pr.pizza_id = co.pizza_id
            ),
            exclusions AS (
            SELECT
            order_id,
            pizza_id,
            UNNEST(STRING_SPLIT(NULLIF(exclusions, ''), ','))::INTEGER as topping_id
            FROM customer_orders
            WHERE exclusions IS NOT NULL AND exclusions != ''
            ),
            extras AS (
            SELECT
            order_id,
            pizza_id,
            UNNEST(STRING_SPLIT(NULLIF(extras, ''), ', '))::INTEGER as topping_id
            FROM customer_orders
            WHERE extras IS NOT NULL AND extras != ''
            ), 
            final_ingredient_ids AS (
            SELECT order_id, pizza_id, topping_id FROM base_toppings
            EXCEPT
            SELECT order_id, pizza_id, topping_id FROM exclusions
            UNION ALL -- allows duplicate rows
            SELECT order_id, pizza_id, topping_id FROM extras
            ),
            topping_counts AS (
            SELECT 
            f.order_id,
            f.pizza_id,
            pt.topping_name,
            COUNT(*) as count
            FROM final_ingredient_ids f
            JOIN pizza_toppings pt ON f.topping_id = pt.topping_id
            GROUP BY f.order_id, f.pizza_id, pt.topping_name
            ),
            
            orders_with_topping_list AS (
            SELECT
            tc.order_id,
            tc.pizza_id,
            STRING_AGG(
            CASE WHEN count = 1 THEN topping_name
            ELSE CAST(count AS TEXT) || 'x' || topping_name
            END, ', ' ORDER BY topping_name) as topping_list
            FROM topping_counts tc
            GROUP BY tc.order_id, tc.pizza_id
            )
            SELECT
            o.order_id,
            pn.pizza_name ||' : ' || o.topping_list as final_output
            FROM orders_with_topping_list o
            JOIN pizza_names pn
            ON pn.pizza_id = o.pizza_id
            """).fetch_df()

,order_id,final_output
0,1,"Meatlovers : BBQ Sauce, Bacon, Beef, Cheese, C..."
1,2,"Meatlovers : BBQ Sauce, Bacon, Beef, Cheese, C..."
2,3,"Meatlovers : BBQ Sauce, Bacon, Beef, Cheese, C..."
3,3,"Vegetarian : Cheese, Mushrooms, Onions, Pepper..."
4,4,"Meatlovers : BBQ Sauce, Bacon, Beef, Chicken, ..."
5,4,"Vegetarian : Mushrooms, Onions, Peppers, Tomat..."
6,5,"Meatlovers : BBQ Sauce, 2xBacon, Beef, Cheese,..."
7,6,"Vegetarian : Cheese, Mushrooms, Onions, Pepper..."
8,7,"Vegetarian : Bacon, Cheese, Mushrooms, Onions,..."
9,8,"Meatlovers : BBQ Sauce, Bacon, Beef, Cheese, C..."


In [114]:
# What is the total quantity of each ingredient used in all delivered pizzas sorted by most frequent first?

con.execute(""" 
            WITH 
            base_toppings AS (
            SELECT 
            co.order_id,
            co.pizza_id,
            UNNEST(STRING_SPLIT(pr.toppings, ','))::INTEGER as topping_id
            FROM customer_orders co
            JOIN pizza_recipes pr ON pr.pizza_id = co.pizza_id
            ),
            exclusions AS (
            SELECT
            order_id,
            pizza_id,
            UNNEST(STRING_SPLIT(NULLIF(exclusions, ''), ','))::INTEGER as topping_id
            FROM customer_orders
            WHERE exclusions IS NOT NULL AND exclusions != ''
            ),
            extras AS (
            SELECT
            order_id,
            pizza_id,
            UNNEST(STRING_SPLIT(NULLIF(extras, ''), ', '))::INTEGER as topping_id
            FROM customer_orders
            WHERE extras IS NOT NULL AND extras != ''
            ), 
            final_ingredient_ids AS (
            SELECT order_id, pizza_id, topping_id FROM base_toppings
            EXCEPT
            SELECT order_id, pizza_id, topping_id FROM exclusions
            UNION ALL -- allows duplicate rows
            SELECT order_id, pizza_id, topping_id FROM extras
            ),
            topping_counts AS (
            SELECT 
            pt.topping_name,
            COUNT(*) as quantity_used
            FROM final_ingredient_ids f
            JOIN pizza_toppings pt ON f.topping_id = pt.topping_id
            GROUP BY pt.topping_name
            )

            SELECT
            *
            FROM topping_counts
            ORDER BY quantity_used DESC
            """).fetch_df()

,topping_name,quantity_used
0,Bacon,12
1,Mushrooms,11
2,Cheese,10
3,Chicken,9
4,Beef,8
5,Salami,8
6,Pepperoni,8
7,BBQ Sauce,7
8,Onions,4
9,Tomato Sauce,4


## Pricing and Ratings

In [6]:
# If a Meat Lovers pizza costs $12 and Vegetarian costs $10 and there were no charges for changes - 
# how much money has Pizza Runner made so far if there are no delivery fees?
con.execute("""
            SELECT 
            SUM(
            CASE WHEN pn.pizza_name = 'Vegetarian' THEN 10
            ELSE 12
            END
            ) as total_revenue
            FROM customer_orders co
            JOIN pizza_names pn
            ON pn.pizza_id = co.pizza_id
            """).fetch_df()

,total_revenue
0,160.0


In [43]:
# What if there was an additional $1 charge for any pizza extras?
# ex: Add cheese is $1 extra

con.execute("""
    SELECT 
        SUM(
            CASE 
                WHEN pn.pizza_name = 'Vegetarian' THEN 10
                ELSE 12
            END
        ) + SUM(
            CASE 
                WHEN co.extras IS NOT NULL AND co.extras <> '' 
                THEN ARRAY_LENGTH(STRING_TO_ARRAY(co.extras, ','), 1)
                ELSE 0
            END
        ) AS total_revenue_with_extras
    FROM customer_orders co
    JOIN pizza_names pn ON pn.pizza_id = co.pizza_id
""").fetch_df()


,total_revenue_with_extras
0,166.0


In [45]:
'''
The Pizza Runner team now wants to add an additional ratings system that allows customers to rate their runner, 
how would you design an additional table for this new dataset - 
generate a schema for this new table and insert your own data for ratings for each successful customer order between 1 to 5.
'''
con.execute(""" 
            CREATE TABLE IF NOT EXISTS ratings 
            (
            order_id INT,
            runner_id INT,
            rating_value INT CHECK (rating_value BETWEEN 1 AND 5), 
            PRIMARY KEY (order_id, runner_id),
            --FOREIGN KEY (order_id) REFERENCES customer_orders(order_id), Although we need to define PK in customer_orders for this to work
            --FOREIGN KEY (runner_id) REFERENCES runners(runner_id)
            );
            """)

con.execute(""" 
    INSERT INTO ratings (order_id, runner_id, rating_value) VALUES 
    (1, 2, 5),  -- Order 1, Runner 2, Rated 5 Stars
    (2, 3, 4),  -- Order 2, Runner 3, Rated 4 Stars
    (3, 1, 3),  -- Order 3, Runner 1, Rated 3 Stars
    (4, 2, 5),  -- Order 4, Runner 2, Rated 5 Stars
    (5, 3, 2)   -- Order 5, Runner 3, Rated 2 Stars
""")

con.execute("SELECT * FROM ratings").fetch_df()

,order_id,runner_id,rating_value
0,1,2,5
1,2,3,4
2,3,1,3
3,4,2,5
4,5,3,2


In [57]:
'''
Using your newly generated table - can you join all of the information together to form a table which has the following information for 
successful deliveries?
customer_id
order_id
runner_id
rating
order_time
pickup_time
Time between order and pickup
Delivery duration
Average speed
Total number of pizzas
'''
con.execute("""
            SELECT
            co.customer_id,
            co.order_id,
            ro.runner_id,
            r.rating_value as rating,
            co.order_time as order_time,
            ro.pickup_time as pickup_time, 
            AVG(DATE_DIFF('minute', co.order_time, CAST(ro.pickup_time AS TIMESTAMP))) as mins_between_order_pickup,
            AVG(CAST(REGEXP_EXTRACT(duration, '[0-9]+(\.[0-9]+)?') AS FLOAT)) as duration_mins,
            (AVG(CAST(REGEXP_EXTRACT(ro.distance, '[0-9]+(\.[0-9]+)?') AS FLOAT))*60)
            /AVG(CAST(REGEXP_EXTRACT(duration, '[0-9]+(\.[0-9]+)?') AS FLOAT)) AS speed_kph,
            COUNT(co.pizza_id) as total_num_of_pizzas
            FROM customer_orders co
            LEFT JOIN ratings r
            ON r.order_id = co.order_id
            JOIN runner_orders ro
            ON ro.order_id = co.order_id
            WHERE ro.pickup_time IS NOT NULL
            GROUP BY ALL
            """).fetch_df()

,customer_id,order_id,runner_id,rating,order_time,pickup_time,mins_between_order_pickup,duration_mins,speed_kph,total_num_of_pizzas
0,101,1,1,5.0,2020-01-01 18:05:02,2020-01-01 18:15:34,10.0,32.0,37.500000,1
1,104,10,1,NaN,2020-01-11 18:34:49,2020-01-11 18:50:20,16.0,10.0,60.000000,2
2,102,8,2,NaN,2020-01-09 23:54:33,2020-01-10 00:15:02,21.0,15.0,93.599998,1
3,105,7,2,NaN,2020-01-08 21:20:29,2020-01-08 21:30:45,10.0,25.0,60.000000,1
4,101,2,1,4.0,2020-01-01 19:00:52,2020-01-01 19:10:54,10.0,27.0,44.444444,1
5,102,3,1,3.0,2020-01-02 23:51:23,2020-01-03 00:12:37,21.0,20.0,40.199999,2
6,103,4,2,5.0,2020-01-04 13:23:46,2020-01-04 13:53:03,30.0,40.0,35.099999,3
7,104,5,3,2.0,2020-01-08 21:00:29,2020-01-08 21:10:57,10.0,15.0,40.000000,1


In [69]:
# If a Meat Lovers pizza was $12 and Vegetarian $10 fixed prices with no cost for extras and each runner is paid $0.30 per kilometre traveled - 
# how much money does Pizza Runner have left over after these deliveries?
con.execute("""
    WITH revenue AS (
        SELECT 
            SUM(
                CASE WHEN pn.pizza_name = 'Vegetarian' THEN 10
                     ELSE 12 
                END
            ) AS total_revenue
        FROM customer_orders co
        JOIN pizza_names pn ON pn.pizza_id = co.pizza_id
    ),
    costs AS (
        SELECT 
            SUM(
                COALESCE(
                    CAST(REGEXP_EXTRACT(distance, '[0-9]+(\.[0-9]+)?') AS FLOAT), 
                    0
                ) * 0.30
            ) AS runner_fee
        FROM runner_orders
        WHERE pickup_time IS NOT NULL  -- Ensures order was delivered
    )
    SELECT 
        r.total_revenue - c.runner_fee AS profit
    FROM revenue r
    CROSS JOIN costs c
""").fetch_df()


,profit
0,116.44
